# Mantid neutron data reduction demo

Welcome to the mantid neutron data demo! Please see the repo `readme` and make sure all prerequisites are met. 

In this tutorial, we will use the mantid framework to load, visualise and manipulate the neutron dataset you've downloaded from dropbox TODO: add link.

Mantid will be invoked using python scripting and some basic familiarity with python is assumed.

## Step one: import mantid and some other useful tools

As with most python scripts, our script will begin with importing the modules we're going to use. To do this, select the cell below and hit `Shift-Enter` to run it.

If you see any errors at this stage, check your conda environment is activated!

In [1]:
from mantid.simpleapi import *
import matplotlib.pyplot as plt
import numpy as np

FrameworkManager-[Notice] Welcome to Mantid 6.10.0
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid6.10
DownloadInstrument-[Notice] Downloading 1 file from the instrument repository


In [3]:
a = np.array([1,2,3])

In [4]:
a

array([1, 2, 3])